In [1]:
%matplotlib inline
from IPython.core.pylabtools import figsize
import matplotlib.pyplot as plt
import scipy.stats as stats

In [ ]:
Soft evidence имитируем монеткой (Bernoulli).

In [23]:
import pymc3 as pm
import numpy as np
import theano.tensor as tt
with pm.Model() as model:
    alpha = pm.Uniform('intercept', -100, 100)

    # Create custom densities
    beta = pm.DensityDist('beta', lambda value: -1.5 * tt.log(1 + value**2), testval=0)
    eps = pm.DensityDist('eps', lambda value: -tt.log(tt.abs_(value)), testval=1)
    print(beta.__dict__)
    lam = pm.Exponential('lam', 1)
    print(lam.__dict__)
    

{'logp_sum_unscaledt': Sum{acc_dtype=float64}.0, 'type': TensorType(float64, scalar), 'index': None, 'model': <pymc3.model.Model object at 0x00000292D24BD0B8>, 'random': None, 'distribution': <pymc3.distributions.distribution.DensityDist object at 0x00000292D244B390>, 'dsize': 1, 'logp_elemwiset': Elemwise{mul,no_inplace}.0, 'name': 'beta', 'owner': None, 'tag': scratchpad{'test_value': 0.0}, 'auto_name': 'auto_24864', 'dshape': (), 'scaling': TensorConstant{1.0}, 'logp_nojac_unscaledt': Elemwise{mul,no_inplace}.0, 'total_size': None}
{'type': TensorType(float64, scalar), 'index': 0, 'distribution': <pymc3.distributions.continuous.Exponential object at 0x00000292D2491198>, 'transformation': <pymc3.distributions.transforms.Log object at 0x00000292CF8465C0>, 'random': <pymc3.model.InstanceMethod object at 0x00000292D22B9B38>, 'transformed': lam_log__, 'model': <pymc3.model.Model object at 0x00000292D24BD0B8>, 'auto_name': 'auto_25233', 'owner': ViewOp(Elemwise{exp,no_inplace}.0), 'tag': 

In [24]:
import pymc3 as pm
import numpy as np
import theano.tensor as tt
with pm.Model() as model:
    lam = pm.Exponential('lam', 1)

    failure = np.array([0, 1])
    value = np.array([1, 0])

    def logp(value):
        return lam * value

    x = pm.DensityDist('x', logp)

In [13]:
import pymc3 as pm
import numpy as np
import theano.tensor as tt

with pm.Model() as model:
    f_1_of_10000 = pm.Categorical("f_1_of_10000", [1, 2])
    print(f_1_of_10000.random())
    def logp(value):
        if (value == 0):
            return 0.33
        elif (value == 1):
            return 0.67
        else:
            return 0
    
    ab = pm.DensityDist('ab', logp)
    modelo=pm.ab(value=0)
    print(modelo.random())
#     samples = [t.random()[0] for i in range(20000)]
#     plt.hist(samples, bins=70, normed=True, histtype="stepfilled")
    
#     # the same as soft evidence fact
#     f_1_of_10000 = pm.Categorical("f_1_of_10000", [0, 1])
#     print(f_1_of_10000.random())
#     I_am_HIV_positive = pm.Categorical("I_am_HIV_positive", [0.0001, 0.9999])
#     I_dont_know = pm.Categorical("I_dont_know", [0.5, 0.5])
#     p_I_am_HIV_positive_soft = pm.math.switch(tt.eq(f_1_of_10000, 1), I_am_HIV_positive, I_dont_know)
    
#     f_HIV_test_false_positive = pm.math.switch(
#         p_I_am_HIV_positive_soft == 0,
#         pm.Categorical("test_if_I_am_not_HIV", [0.999, 0.001]),
#         pm.Categorical("test_if_I_am_HIV", [0.001, 0.999]),
#         observed=1
#     )

1


AttributeError: module 'pymc3' has no attribute 'ab'